In [ ]:
pip install torchviz

In [ ]:
pip install hiddenlayer

In [ ]:
# Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tarfile
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import time
from torchviz import make_dot
import hiddenlayer as hl
from collections import Counter
from imblearn.over_sampling import SMOTE

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import transforms
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

#volatille
#b06bccd3d1795d29ff0b071f635f2d79

import os
os.environ['KAGGLE_USERNAME'] = "volatille"
os.environ['KAGGLE_KEY'] = "b06bccd3d1795d29ff0b071f635f2d79"
!kaggle datasets download  paultimothymooney/breast-histopathology-images
#od.download("https://www.kaggle.com/paultimothymooney/breast-histopathology-images", force=True, username='volatille', key='b06bccd3d1795d29ff0b071f635f2d79') #Wczesniej wczytywałem poprzez sciezkie w systemie, po migracji na coolab zmieniłem ścieżkę.



In [ ]:
#Rozpakowywuje baze danych, ukrywam output, bo za dużo śmieci

!unzip breast-histopathology-images.zip

In [ ]:
#Tworzenie bazy

data_dir = './'
folder_name = "IDC_regular_ps50_idx5"
image_folders = os.path.join(data_dir, folder_name)

transform = transforms.Compose([transforms.Resize((50, 50)), transforms.ToTensor()]) #Funkcja robie resize każdego pliku (50x50) oraz rzutuję na = torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] 

zdjecia_idc = []
for file in os.listdir(image_folders):
    zdjecia_idc.append(ImageFolder(os.path.join(image_folders, file), transform=transform)) #pętla, alokacja wczesniej przyjętych zmian dla plików.
datasets = torch.utils.data.ConcatDataset(zdjecia_idc) #upchanie wszystkiego do datasets

In [ ]:
from collections import Counter

#Obliczanie ile jest danych w zbiorze (IDC)
j=0
for dataset in tqdm(datasets.datasets):
    if j == 0:
        result = Counter(dataset.targets)
        j += 1
    else:
        result += Counter(dataset.targets)


Pliki_2 = os.listdir(os.path.join(data_dir, folder_name))
print(" \n Liczba Pacjentów:", len(Pliki_2))

print("""\n Liczba zdjęć:
    IDC_0 (Brak IDC)  : {}
    IDC_i (Obecne IDC): {}""".format(result[0], result[1]))


In [ ]:
# Przygotowywanie modelu,danych itp.
random_seed = 42
torch.manual_seed(random_seed)

test_size = 38000
train_size = len(datasets) - test_size
train_ds, test_ds = random_split(datasets, [train_size, test_size])

val_size = 38000
train_size = len(train_ds) - val_size
train_ds, val_ds = random_split(train_ds, [train_size, val_size])

len(train_ds), len(val_ds), len(test_ds)

In [ ]:
#Dane do treningu, validacji oraz testu
#   shuffle=True, przy false może nastąpić większy błąd val_loss (w treningu sieci)
#   pin_memory to wstawka dla conda cpu

train_data = DataLoader(train_ds, shuffle=True, num_workers=4, pin_memory=True)
val_data = DataLoader(val_ds, shuffle=True, num_workers=4, pin_memory=True)
test_data = DataLoader(test_ds, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
# Zapis danych dysk

def save_data(data, mode="train"):
    i = 0
    for img, label in data:
        folder_path = os.path.join(os.path.join(os.getcwd(), mode), str(np.array(label)[0]))
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        i += 1
        image = transforms.ToPILImage()(img[0, :, :, :])
        image.save(os.path.join(folder_path, mode+"_"+str(i)+".jpg"), "JPEG")

In [ ]:

#Save Train Data
save_data(train_data, mode="train")

# Save Validation Data
save_data(val_data, mode="validation")

# Save Test Data
save_data(test_data, mode="test")

In [ ]:
# Transformacja danych /tt.ToTensor 
train_tfms = tt.Compose([tt.ToTensor()])
valid_tfms = tt.Compose([tt.ToTensor()])

In [ ]:
data_dir = "./"

train_file = os.path.join(data_dir, "train")
val_file = os.path.join(data_dir, "validation")
test_file = os.path.join(data_dir, "test")

train_ds = ImageFolder(train_file, train_tfms)
val_ds = ImageFolder(val_file, valid_tfms)
test_ds = ImageFolder(test_file, valid_tfms)

In [ ]:
#     Batch_size - liczba próbek propagowanych przez uczącą się sieć. Czyli jesli równe 1000, to pobiera 1000 uczy sieć, potem znów pobiera 1000-2000 (1k) i uczy sieć.
#     problemem z batch size może być liczba próbek niepodzielna przez parametr ustawiony przez nas.


batch_size = 100

In [ ]:
# Data loadery. Podajemy dane dla treningu, validacji, testu.
#     Numworkers = 3, tutaj experymentalnie 3. 
#     Numworkers = 0 to znaczy że jest to główny proces
#     Numworkers = 1, oznacza że proces podzielony i ma jednego "podwykonawcę" (wedługp rzeczytanych przykładów, zazwyczaj tyle wstarczy do poprawnej implementacji)
#     Im więcej num_workers tym większe zasoby zuzywamy, a często nic nie zyskujemy. Możliwa obserwacja i wyciągnięcie wniosków poprzez wartości train_loss oraz val_loss

#     Shuffle, tasujemy dane treningowe.

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=3, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size*2, num_workers=3, pin_memory=True)

In [ ]:
### Funkcja wskaźnika F1 według algorytmu ###
#   Funkcja jest testem na dokładność. Wartość 1.0 oznacza najwyższą precyzję

#Funkcja oblicza wartości:
# TP = True Positives
# TN = True Negatives
# FP = False Positives
# FN = False Negatives

def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [ ]:
#Klasa PyTorch zawierająca funkcję do kroku treningu modułu, kroku validacji modułu.
# @training_step() zwraca train_loss
# @validation_step() zwraca validation_loss
# @validation_epoch_end() 


class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, targets = batch 
        targets = torch.reshape(targets.type(torch.cuda.FloatTensor), (len(targets), 1))
        out = self(images)                      
        loss = F.binary_cross_entropy(out, targets)      
        return loss
    
    def validation_step(self, batch):
        images, targets = batch
        targets = torch.reshape(targets.type(torch.cuda.FloatTensor), (len(targets), 1))
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))

In [ ]:
class BreastCancerResnet34(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, 1)
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.fc.parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
test_dl = DeviceDataLoader(test_dl, device)

In [ ]:
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
model = to_device(BreastCancerResnet34(), device)

In [ ]:
history = [evaluate(model, val_dl)]
history

In [ ]:
#Resnet z metodą nauki 1 warstwy z zamrożeniem

model.freeze()

In [ ]:
epochs = 15
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
%%time
start_time = time.time()
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

train_time = time.time() - start_time

In [ ]:
model.unfreeze()

In [ ]:
%%time
start_time = time.time()
history += fit_one_cycle(epochs, 0.001, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)
train_time += time.time() - start_time

In [ ]:
#Blokujemy gradient, obliczanie pred. danych

@torch.no_grad()
def predict_dl(dl, model, threshold=0.5):
    torch.cuda.empty_cache()
    batch_probs = []
    for xb, _ in tqdm(dl):
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    return [int(x) for x in batch_probs>threshold]

In [ ]:
# Analyze Prediction Results
test_preds = predict_dl(test_dl, model)
actual_label = test_dl.dl.dataset.targets

In [ ]:
#Macierz błedów. Obliczanie

f1 = f1_score(actual_label, test_preds)
f_score = float(np.array(F_score(torch.tensor(np.array(test_preds).reshape(len(test_preds), 1)), torch.tensor(np.array(actual_label).reshape(len(actual_label), 1)))))
accuracy = accuracy_score(actual_label, test_preds)
cm = confusion_matrix(actual_label, test_preds)
report = classification_report(actual_label, test_preds)

print("Model F-Score (Test Data): ", f_score)
print("Model F1-Score (Test Data): ", f1)
print("Model Accuracy: ", accuracy)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", report)

# Plot Confusion Matrix
df_cm = pd.DataFrame(cm, index = [i for i in "01"], columns = [i for i in "01"])
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)
sns.heatmap(df_cm, cmap="Oranges", annot=True, annot_kws={"size": 16})
plt.title("Plot of Confusion Matrix")
plt.show()
plt.savefig("ResNet34_CM")